In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!nvidia-smi

Tue Jun 25 14:24:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   34C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install datasets
!pip install trl
!pip install bitsandbytes
!pip install peft
!pip install flash-attn --no-build-isolation
!pip install mlflow
!pip install torchinfo
!pip install databricks-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [3]:
from datasets import load_dataset, load_from_disk
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM, PeftModel, PeftConfig, PeftModelForCausalLM
import gc
import mlflow
from datetime import datetime
import re
import os

In [4]:
mlflow.login()

2024/06/30 10:36:30 INFO mlflow.utils.credentials: No valid Databricks credentials found, please enter your credentials...


Databricks Host (should begin with https://): https://community.cloud.databricks.com
Username: shaunakketkar@gmail.com
Password: ··········


2024/06/30 10:36:49 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://community.cloud.databricks.com.


In [1]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(experiment_id=os.environ.get("MLFLOW_EXPERIMENT_ID"))

NameError: name 'mlflow' is not defined

In [2]:
model_base_path = "./models/Model_weights"

max_count = 0
for model_dir in os.listdir(model_base_path):
  match_obj = re.search(r"phi-2_sft_v(\d{1,3})", model_dir)
  if(match_obj):
    max_count = max(max_count, int(match_obj.group(1)))

new_model_num = max_count + 1
new_model_path = os.path.join(model_base_path, f"phi-2_sft_v{new_model_num}")
new_final_model_path = os.path.join(model_base_path, f"phi-2_sft_final_v{new_model_num}")

NameError: name 'os' is not defined

In [7]:
dataset = load_from_disk("./datasets/SlimOrca-Transformed-with-eos")

In [8]:
tokenizer = AutoTokenizer.from_pretrained("./models/Model_weights/phi-2", padding_side="left")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
class SpecialTokens:
    conversation_start_token: str = "<|im_start|>"
    conversation_end_token: str = "<|im_end|>"
    pad_token: str = "<|pad|>"
    system_token = "<|system|>"
    user_token = "<|user|>"
    assistant_token = "<|assistant|>"


chat_format_tokens = SpecialTokens

# Add special tokens to the tokenizer
tokenizer.add_special_tokens(
    {
        "additional_special_tokens": [
            chat_format_tokens.conversation_start_token,
            chat_format_tokens.conversation_end_token,
            chat_format_tokens.pad_token,
            chat_format_tokens.system_token,
            chat_format_tokens.user_token,
            chat_format_tokens.assistant_token
        ]
    }
)

CHAT_TEMPLATE = """{% for message in messages %}{{'<|im_start|>' + '<|' + message['role'] + '|>' + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"""
tokenizer.chat_template = CHAT_TEMPLATE
tokenizer.pad_token = chat_format_tokens.pad_token

In [12]:
print(tokenizer.vocab["<|im_start|>"])
print(tokenizer.vocab["<|im_end|>"])
print(tokenizer.vocab["<|system|>"])
print(tokenizer.vocab["<|user|>"])
print(tokenizer.vocab["<|assistant|>"])

50295
50296
50298
50299
50300


In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_quant_type='nf4',
)

model = AutoModelForCausalLM.from_pretrained(
    "./models/Model_weights/phi-2",
    device_map=torch.device(0),
    torch_dtype=torch.float32,
    trust_remote_code=True,
    quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50301, 2560)

In [15]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "dense"
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
    modules_to_save=["embed_tokens", "lm_head"]
)

model.gradient_checkpointing_enable()

training_arguments = TrainingArguments(
    output_dir="./Results/mlflow_v2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    max_grad_norm=.3,
    max_steps=30,
    warmup_ratio=.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    num_train_epochs=1,
    log_level="error",
    logging_strategy="steps",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    report_to="mlflow",
    run_name=f"phi-2-sft-LORA-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}"
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    peft_config=lora_config,
    packing=False,
    max_seq_length=720
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the

Map:   0%|          | 0/517982 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [16]:
trainer.train()

Step,Training Loss
1,2.387300
2,2.119300
3,2.221200
4,2.195300
5,2.103400
6,2.143300
7,2.183300
8,2.222000
9,2.103700
10,2.262200


TrainOutput(global_step=30, training_loss=1.8317962527275085, metrics={'train_runtime': 631.0131, 'train_samples_per_second': 0.761, 'train_steps_per_second': 0.048, 'total_flos': 3873048482871120.0, 'train_loss': 1.8317962527275085, 'epoch': 0.0009266695496385988})

In [17]:
last_run_id = mlflow.last_active_run().info.run_id
print(last_run_id)

127eefa63b414f9d9e77edc0e5c53d2e


In [18]:
with mlflow.start_run(run_id=last_run_id):
  mlflow.log_params(lora_config.to_dict())
  mlflow.transformers.log_model({"model": trainer.model, "tokenizer": tokenizer}, artifact_path="model", task="text-generation")

<ipython-input-18-d6c31d9dc355>:3: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.41.0``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlflow.transformers.log_model({"model": trainer.model, "tokenizer": tokenizer}, artifact_path="model", task="text-generation")
/usr/local/lib/python3.10/dist-packages/mlflow/models/model.py:684: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.41.0``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2024/06/30 11:04:49 INFO mlflow.transformers: Overriding save_pretrained to False for PEFT models, following the Transformers behavior. The PEFT adaptor

Uploading artifacts:   0%|          | 0/33 [00:00<?, ?it/s]

Uploading /tmp/tmpiq0kq5kl/model/model/model-00001-of-00010.safetensors:   0%|          | 0.00/491M [00:00<?, …

Uploading /tmp/tmpiq0kq5kl/model/model/model-00007-of-00010.safetensors:   0%|          | 0.00/491M [00:00<?, …

Uploading /tmp/tmpiq0kq5kl/model/model/model-00009-of-00010.safetensors:   0%|          | 0.00/491M [00:00<?, …

Uploading /tmp/tmpiq0kq5kl/model/model/model-00002-of-00010.safetensors:   0%|          | 0.00/491M [00:00<?, …

Uploading /tmp/tmpiq0kq5kl/model/peft/adapter_model.safetensors:   0%|          | 0.00/1.04G [00:00<?, ?iB/s]

In [19]:
trainer.save_model(new_model_path)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /content/gdrive/MyDrive/Colab Notebooks/LLMs/Models/phi-2 - will assume that the vocabulary was not modified.
  warnings.warn(


In [20]:
del trainer
del model
gc.collect()
torch.cuda.empty_cache()

In [21]:
# If model embedding resizing is done prior
model = AutoPeftModelForCausalLM.from_pretrained(
    new_model_path,
    adapter_name="sft",
    low_cpu_mem_usage=True,
    device_map="auto",
    trust_remote_code=True,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    new_final_model_path,
    safe_serialization=True,
    max_shard_size="2GB",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# If model embedding resizing is not done prior and if del model is executed then model can be reloaded in the same way we
# first load it before training, just run the cell above used for loading fresh Phi-2 model
new_model = PeftModel.from_pretrained(
    model,
    new_model_path,
    adapter_name="sft",
    low_cpu_mem_usage=True,
    device_map="auto",
    trust_remote_code=True
)

merged_model = new_model.merge_and_unload()
merged_model.save_pretrained(
    new_final_model_path,
    safe_serialization=True,
    max_shard_size="2GB",
)

AttributeError: 'PhiForCausalLM' object has no attribute 'merge_and_unload'